# Introduction to Python and Natural Language Technologies

__Laboratory 09, Transformers__

__April 14, 2020__

The starter code in this notebook is the same as in Assignment 8. The number of training epochs is increased.

Your task is to replace the model with a Transformer.

Transformer is known to perform worse than LSTMs on small datasets so do not be alarmed when you see a drop in performance.

If you successfully solved Assignment 8, you may start by replacing the starter code with your implementation.

Passing level: Task 1, 2

Extra level: Task 3, 4

## Task 1 - Replace LSTMClassifier with a Transformer-based encoder without positional encoding and attention masking.

We suggest reading this [tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html) first.
The tutorial is for a sequence-to-sequence model which can be easily adapted to sequence classification.
You can use the first output of the Transformer as the representation of the full word.

Make sure that the Transformer parameters are not hardwired.

Don't forget to rename the class since it's no longer an 'LSTM' classifier.

Try out a few options for the Transformer parameters and summarize your findings in a few sentences in the following cell.

YOUR ANSWER HERE

## Task 2 - Positional encoding

Add positional encoding to the Transformer. 
You can take inspiration from Lecture 9.

Make the base of the positional encoding (10000) by default configurable.

Try out a few options and summarize your findings in the following cell.

YOUR ANSWER HERE

## Task 3 - Attention masking

The Transformer does not need to put any attention weight on PAD symbols.
This can be done by supplying a mask to `TransformerEncoder`.

Add this mask and summarize your findings in the following cell.

YOUR ANSWER HERE

## Task 4 - Training improvement

Transformers are notoriously hard to train.

Try to improve the training process.
Here is an incomplete list of tricks that may improve the training process:
- learning rate decay
- different initialization
- warm up.

You are encouraged to find further tricks.

Try at least two tricks and summarize your findings. It does not matter if they do not improve the accuracy of the classifier.

YOUR ANSWER HERE

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim

# Cloning the data repository

In [ ]:
language = "hun"
unimorph_path = f"data/unimorph_{language}/"
pipe = subprocess.Popen(f"git clone git@github.com:unimorph/{language}.git {unimorph_path}",
                        shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
stdout, stderr = pipe.communicate()
print(stdout.decode('utf8'))
print(stderr.decode('utf8'))

# Data loading

In [ ]:
data = pd.read_table(f"{unimorph_path}/{language}", names=['lemma', 'infl', 'tags'], skip_blank_lines=True)

# Target extraction

In [ ]:
def extract_target(tags_str):
    """Extracts target if present, returns None otherwise."""
    tags = tags_str.split(";")
    if tags[0] != 'V':
        return None
    if len(tags) < 6:
        return None
    return tags[1]
 
data['target'] = data.tags.apply(extract_target)

In [ ]:
data = data[data.target.notnull()]

In [ ]:
data['target'].value_counts()

# Train/dev/test set creation

We avoid lemma overlaps between the splits.

In [ ]:
lemmas = data.lemma.unique()
len(lemmas), type(lemmas)

In [ ]:
np.random.seed(12)
np.random.shuffle(lemmas)
train_size = int(0.8 * len(lemmas))
dev_size = int(0.1 * len(lemmas))
train_lemmas = lemmas[:train_size]
dev_lemmas = lemmas[train_size:train_size+dev_size]
test_lemmas = lemmas[train_size+dev_size:]

train_lemmas = set(train_lemmas)
dev_lemmas = set(dev_lemmas)
test_lemmas = set(test_lemmas)

In [ ]:
train_df = data[data.lemma.isin(train_lemmas)]
dev_df = data[data.lemma.isin(dev_lemmas)]
test_df = data[data.lemma.isin(test_lemmas)]
len(train_df), len(dev_df), len(test_df)

In [ ]:
train_df = train_df.sample(1000, random_state=1).reset_index(drop=True)
dev_df = dev_df.sample(200, random_state=1).reset_index(drop=True)
test_df = test_df.sample(200, random_state=1).reset_index(drop=True)

# Preprocessing

In [ ]:
alphabet = set()
for token in train_df.infl:
    alphabet |= set(token)
len(alphabet)

In [ ]:
alphabet.add('<PAD>')
alphabet.add('<BOS>')
alphabet.add('<EOS>')
alphabet.add('<UNK>')

In [ ]:
vocab = {symbol: i for i, symbol in enumerate(alphabet)}
len(vocab)

In [ ]:
def encode_token(token):
    ids = []
    ids.append(vocab['<BOS>'])
    # dev and test might contain characters outside the alphabet
    ids.extend(vocab.get(c, vocab['<UNK>']) for c in token)
    ids.append(vocab['<EOS>'])
    return ids

print(f"{encode_token('alma') = }")
print(f"{vocab['<UNK>'] = }")
print(f"{encode_token('ALMA') = }")

In [ ]:
train_df['encoded'] = train_df.infl.apply(encode_token)
dev_df['encoded'] = dev_df.infl.apply(encode_token)
test_df['encoded'] = test_df.infl.apply(encode_token)
train_df.head()

In [ ]:
maxlen = train_df.encoded.apply(len).max()
print(maxlen)

def pad_sequence(sequence):
    if len(sequence) > maxlen:
        return sequence[:maxlen]
    return sequence + [vocab['<PAD>'] for _ in range(maxlen-len(sequence))]

print(pad_sequence([1, 2, 3]))

In [ ]:
train_df['padded'] = train_df.encoded.apply(pad_sequence)
dev_df['padded'] = dev_df.encoded.apply(pad_sequence)
test_df['padded'] = test_df.encoded.apply(pad_sequence)

train_df['padded'].apply(len).value_counts()

In [ ]:
train_df['seqlen'] = train_df.encoded.apply(len)
dev_df['seqlen'] = dev_df.encoded.apply(len)
test_df['seqlen'] = test_df.encoded.apply(len)

In [ ]:
label_to_id = {label: i for i, label in enumerate(train_df.target.unique())}
label_to_id

In [ ]:
train_df['label'] = train_df.target.apply(lambda c: label_to_id[c])
dev_df['label'] = dev_df.target.apply(lambda c: label_to_id[c])
test_df['label'] = test_df.target.apply(lambda c: label_to_id[c])

# Creating tensors

In [ ]:
X_train = torch.from_numpy(np.array(list(train_df.padded)))
y_train = torch.LongTensor(train_df.label.values)
seqlen_train = torch.LongTensor(train_df.seqlen.values)
print(f"{X_train.size() = },\n{y_train.size() = }\n{seqlen_train.size() = }\n")

X_dev = torch.from_numpy(np.array(list(dev_df.padded)))
y_dev = torch.LongTensor(dev_df.label.values)
seqlen_dev = torch.LongTensor(dev_df.seqlen.values)
print(f"{X_dev.size() = },\n{y_dev.size() = }\n{seqlen_dev.size() = }\n")

X_test = torch.from_numpy(np.array(list(test_df.padded)))
y_test = torch.LongTensor(test_df.label.values)
seqlen_test = torch.LongTensor(test_df.seqlen.values)
print(f"{X_test.size() = },\n{y_test.size() = }\n{seqlen_test.size() = }")

# Model

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size):
        super().__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            num_layers=1,
            bidirectional=True,
            batch_first=True,
        )
        self.dense = nn.Linear(hidden_size * 2, output_size)
        
    # the input signature of forward changes
    def forward(self, sequences, sequence_lens):
        embedded = self.embedding(sequences)
        
        # THIS IS THE MODIFIED PART
        # returns a PackedSequence object
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded,
            sequence_lens,
            enforce_sorted=False,
            batch_first=True)
        packed_outputs, (h, c) = self.lstm(packed)
        # extract LSTM outputs (not used here)
        lstm_outputs, lens = nn.utils.rnn.pad_packed_sequence(packed_outputs)
        
        h = torch.cat((h[0], h[1]), dim=-1)
        output = self.dense(h)
        return output

In [ ]:
input_size = len(vocab)
embedding_size = 30
hidden_size = 64
output_size = train_df.label.nunique()

model = LSTMClassifier(input_size, embedding_size, hidden_size, output_size)
model

# Batching

In [ ]:
class BatchedIterator:
    def __init__(self, *tensors, batch_size):
        # all tensors must have the same first dimension
        assert len(set(len(tensor) for tensor in tensors)) == 1
        self.tensors = tensors
        self.batch_size = batch_size
    
    def iterate_once(self):
        num_data = len(self.tensors[0])
        for start in range(0, num_data, self.batch_size):
            end = start + self.batch_size
            yield tuple(tensor[start:end] for tensor in self.tensors)

# Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
num_epochs = 50
batch_size = 128

metrics = defaultdict(list)
train_iter = BatchedIterator(X_train, seqlen_train, y_train, batch_size=batch_size)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    # Training loop
    for X_batch, seqlen_batch, y_batch in train_iter.iterate_once():
        y_out = model(X_batch, seqlen_batch)
        loss = criterion(y_out, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()  # or model.train(False)
    # Train and dev loss at the end of the epoch
    y_out = model(X_train, seqlen_train)
    train_loss = criterion(y_out, y_train).item()
    metrics['train_loss'].append(train_loss)
    labels = y_out.argmax(axis=1)
    train_accuracy = (torch.eq(y_train, labels).sum() / float(labels.size(0))).item()
    metrics['train_accuracy'].append(train_accuracy)
    
    y_out = model(X_dev, seqlen_dev)
    dev_loss = criterion(y_out, y_dev).item()
    metrics['dev_loss'].append(dev_loss)
    labels = y_out.argmax(axis=1)
    dev_accuracy = (torch.eq(y_dev, labels).sum() / float(labels.size(0))).item()
    metrics['dev_accuracy'].append(dev_accuracy)
    
    print(f"Epoch: {epoch} -- train loss: {train_loss:.3f} - train acc: {train_accuracy:.1%} - "
          f"dev loss: {dev_loss:.3f} - dev acc: {dev_accuracy:.1%}")

# Evaluation

## Plot the results

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 4))

sns.lineplot(data=metrics['train_loss'], ax=ax[0], label='train loss')
sns.lineplot(data=metrics['dev_loss'], ax=ax[0], label='dev loss')

sns.lineplot(data=metrics['train_accuracy'], ax=ax[1], label='train acc')
sns.lineplot(data=metrics['dev_accuracy'], ax=ax[1], label='dev acc')

## Test accuracy

In [ ]:
logits = model(X_test, seqlen_test)
test_prediction = logits.argmax(axis=1)
test_accuracy = torch.sum(torch.eq(test_prediction, y_test)) / float(test_prediction.size(0))
print(f"Test accuracy: {test_accuracy:.1%}")

## Error analysis

In [ ]:
test_df['prediction'] = test_prediction

In [ ]:
id_to_label = {i: l for l, i in label_to_id.items()}
test_df['predicted_target'] = test_df['prediction'].apply(lambda id_: id_to_label[id_])

In [ ]:
incorrect = test_df[test_df.prediction != test_df.label][['infl', 'target', 'predicted_target']]
incorrect.sample(min(len(incorrect), 20))